<a href="https://colab.research.google.com/github/yutsutke/practice/blob/main/open_youtubepra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client pandas gspread gspread_dataframe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 新しいセクション

# 新しいセクション

In [ ]:
# Youtubeから検索したいワードに基づいた情報（title	views	公開日）のスプレッドシートをグーグルドライブ上に作成
# グーグルコラボ上で実行することを想定してます。
# Google Sheetsへの認証のため、'グーグルコンソール'の'IAMと管理'より'サービス アカウント'を取得し、'credentials.json'を作成し、グーグルドライブ上に置く必要あり
# データを書き込むスプレッドシートは、'スプレッドシートの公開'の'アクセスできるユーザ'にサービスアカウントのメールアドレスを登録しておく必要があります。
# 'Google Sheets API'と'YouTube Data API v3'はAPIのステータスは有効にしてください。
# 入力の必要ある項目は、Google API情報（your key） 検索条件の設定(検索したいワード) スプレッドシートを開く（スプレッドシートＩＤ）スプレッドシート

import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google.oauth2 import service_account
from google.colab import drive

# Google Driveをマウントする
drive.mount('/content/drive')

# Google API情報
api_key = 'your key'
youtube = build('youtube', 'v3', developerKey=api_key)

# 検索条件の設定
search_query = '検索したいワード'
max_results = 50

# 検索結果の取得
search_response = youtube.search().list(
    q=search_query,
    type='video',
    part='id,snippet',
    maxResults=max_results
).execute()

# 動画IDのリストを作成
video_ids = []
for search_result in search_response.get('items', []):
    video_ids.append(search_result['id']['videoId'])

# 動画の統計情報を取得
stats = []
for i in range(0, len(video_ids), 50):
    res = youtube.videos().list(
        id=','.join(video_ids[i:i + 50]),
        part='id,statistics'
    ).execute()
    stats += res['items']

# データの整形
video_data = []
for stat in stats:
    video_id = stat['id']
    views = int(stat['statistics']['viewCount'])
    title = search_response['items'][video_ids.index(video_id)]['snippet']['title']
    video_data.append([title, views])

# データフレームの作成
df = pd.DataFrame(video_data, columns=['title', 'views'])
df = df.sort_values(by='views', ascending=False)

# 公開日の項目を追加する列を選択する
df['公開日'] = ''
for i, item in enumerate(search_response['items']):
    video_id = item['id']['videoId']
    video = youtube.videos().list(part='snippet', id=video_id).execute()['items'][0]
    df.at[i, '公開日'] = video['snippet']['publishedAt']

# Google Sheetsへの認証
creds = service_account.Credentials.from_service_account_file('/content/drive/MyDrive/credentials.json', scopes=['https://www.googleapis.com/auth/spreadsheets'])
gc = gspread.authorize(creds)

# スプレッドシートを開く
sheet_id = 'スプレッドシートＩＤ'
sh = gc.open_by_key(sheet_id)

# シートを指定する

worksheet_name = 'Sheet1'
worksheet = sh.worksheet(worksheet_name)

# データを書き込む
set_with_dataframe(worksheet, df)
